# GENESIS' ORIGINAL CODE

# OBJECTIVE: (TO MY BEST UNDERSTANDING)
1. Build a Recommender System
- Recommender system for matching job descriptions to resumes
- recommdner system that suggests jobs to candidates based on resumes
- build recommender system for matching job descriptions for gov databases
2. Do something with government data
3. Build an app


Key Methods:
- embedding-based recommendations
- fine-tuned transformer model

## code reactions

- embedding models review needed
- review langchain embeddings library


# Dez Next Steps

1. Reproduce and run Genesis code with redacted pii data
2. Add additional LLMs
3. Figure Out How to make the prompt engineering run quicker, maybe do 1 resume at a time then slowly increase the dataset size
4. Figure Out How difficult is it to get api access to the suggested prompt engineering models

Scraped Resumes and Prompt Enegineering (GR Version)
###LLMs used: GPT4, Cohere and Llama 2
###Date: 10/30/2024
###Saxa 4#

**Data source:**  Used "modified_file 1.csv” which is the 2,483 resumes that was scraped online. This file was cleaned up into a “more_resume.json” fine. I used this file as a case study for prompt engineering.
o	Note: This code does NOT incorporate PII identification and redaction, which is Dezmond’s code.


**Key Highlights of	Prompt engineering steps:**

*Step 1.* --> Embedding: Used Hugging Face "all-mpnet-base-v2" as a pre-trained sentence transformer base model designed for generating sentence embeddings (numerical represetnations of sentences that capture semantics (i.e finance, HR, technology etc).

*Step 2* -->Prompt Engineering- started shell of code, but did not get code to run as step 1 above took 2+ hours and was still running.

*   Prompt Engineering 1: Open AI's GPT 4
      -  Updated code to call GR's specific API
      - Updated query prompts for resumes, but did not see output yet
*   Prompt Engineering 2:  Cohere - need to add in Specific API
*   Prompt Engineering 3:  Llama 2- Requested user access request to Meta via Hugging Face, awaiting access approval




***

In [ ]:

!pip install python-docx==1.1.0
!pip install pdfplumber==0.9.0

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from docx import Document
import os
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pdfplumber
import nltk
nltk.download('punkt')
from textblob import TextBlob
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

***

## Adding in the Other Resumes for Prompt Engineering exercise




In [ ]:
more_resumes = pd.read_csv('/content/drive/My Drive/Capstone(Saxa 4)/04Code/modified_file 1.csv')

In [ ]:
more_resumes.to_json('more_resumes.json', orient = 'records', lines = True)

In [ ]:
more_resumes.rename(columns = {'Resume_str': 'text'}, inplace = True)

In [ ]:
more_resumes

In [ ]:
more_resumes = pd.DataFrame(more_resumes)
more_resumes

In [ ]:
more_resumes.dtypes

In [ ]:
#more_resumes['text'] = more_resumes['text'].apply(lambda x: x.encode('utf-8').decode('utf-8'))

In [ ]:
more_resumes.dtypes

In [ ]:
#more_resumes.drop(columns = ['Resume_html'], inplace = True)

In [ ]:
more_resumes

In [ ]:
more_resumes.columns = more_resumes.columns.str.strip()

In [ ]:
more_resumes['text'] = more_resumes['text'].apply(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
more_resumes['text'] = more_resumes['text'].str.lower()

In [ ]:
more_resumes

In [ ]:
more_resumes['text'] = more_resumes['text'].str.replace('[^\w\s]', '', regex=True)

In [ ]:
cats_more_res = more_resumes['Category'].nunique()
cats_more_res

In [ ]:
cat_counts = more_resumes['Category'].value_counts()    #Look at CSV and get get rid of the 1's
cat_counts

In [ ]:
finance = more_resumes[more_resumes['Category'] == 'FINANCE']

In [ ]:
engineering = more_resumes[more_resumes['Category'] == 'ENGINEERING']

In [ ]:
more_resumes

In [ ]:
!pip install langchain==0.0.240 transformers sentence_transformers datasets faiss-cpu

### PROMPT ENGINEERING ##

In [ ]:
#import libraries

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document # Import the Document class
from datasets import load_dataset


# Sentence Transformers

In [ ]:
# 1. Load the Hugging Face Embeddings model
  ##Notes##
   # Code Purpose: loads a pre-trained sentence transformer model called "all-mpnet-base-v2" from Hugging Face.
   # Model insight: "all-mpnet-base-v2" is used to convert text (in this case, resume content) into numerical vectors called embeddings.
   # Functionality: Embeddings capture the semantic meaning of the text, allowing you to compare the similarity between different pieces of text based on their vector representations.
embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")

# 2. Load the more_resumes dataset
  # Assumption about dataset: Assuming 'more_resumes.json' is in the current directory
more_resumes = load_dataset('json', data_files='more_resumes.json')

# 3. Split resumes into smaller chunks for embedding
  #Code purpose: Divides each resume's text content into smaller chunks to handle potentially large documents.
  # Functionality: RecursiveCharacterTextSplitter: This object is used to split the text. It's configured to create
  # chunks of approximately 1000 characters with an overlap of 200 characters between chunks.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = []
for resume in more_resumes['train']:
    texts = text_splitter.split_text(resume['text'])
    # Create Document objects instead of dictionaries
    docs.extend([Document(page_content=t, metadata={'source': resume['Category']}) for t in texts])

# 4. Embed the resume chunks and store them in a FAISS vectorstore
  #Purpose: Purpose: Creates a vector database using FAISS (Facebook AI Similarity Search) and
  # stores the embeddings of the resume chunks in it.
db = FAISS.from_documents(docs, embeddings)
db.save_local("more_resumes_faiss")

# Now the 'more_resumes' dataset is embedded and stored in a FAISS vectorstore.

In [ ]:
###Prompt Engineering 1: Open AI's GPT 4 ###

from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import os        #for API Key
import openai


# 1. Load the FAISS vectorstore
db = FAISS.load_local("more_resumes_faiss", embeddings)

# 2. Initialize OpenAI's GPT-4
# Make sure you set your OPENAI_API_KEY environment variable

llm = OpenAI(model_name="gpt-4", temperature=0)

# Set's GR's API key as an environment variable (recommended)
os.environ["genesis_gpt4_API_key"] = "genesis_gpt4_API_key" #Replace YOUR_API_KEY with your actual OpenAI key

# 3. Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

# 4. Define your prompt template
prompt_template = """
Use the following pieces of context to answer the question at the end.
If you don't have enough information to answer the question, just say "I don't know".

Context:
{context}

Question:
{question}
"""

# 5. Function to run a query
def run_query(query):
    """Runs a query against the resume dataset using prompt engineering."""
    response = qa_chain.run(
        prompt_template.format(context=db.as_retriever().get_relevant_documents(query)[0].page_content, question=query)
    )
    return response #Corrected to return the response

# 6a. Example usage 1
query = "Based on this resume, what are the top jobs I can qualify for?"
result = run_query(query)
print(result)

# 6b. Example usage 2
query1 = "I live in a high cost city and need to make at least $80k salary per year, based on my resume, what jobs should I apply for to get me this salary?"
result1 = run_query(query1)
print(result1)

In [ ]:
###Prompt Engineering 2:  Cohere ###

from langchain.chains import RetrievalQA
from langchain.llms import Cohere

# 1. Load the FAISS vectorstore
db = FAISS.load_local("more_resumes_faiss", embeddings)

# 2. Initialize Cohere
# Make sure you set your COHERE_API_KEY environment variable
llm = Cohere(model="command-xlarge-nightly", temperature=0)

# 3. Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

# 4. Define your prompt template
prompt_template = """
Use the following pieces of context to answer the question at the end.
If you don't have enough information to answer the question, just say "I don't know".

Context:
{context}

Question:
{question}
"""

# 5. Function to run a query
def run_query(query):
    """Runs a query against the resume dataset using prompt engineering."""
    response = qa_chain.run(
        prompt_template.format(context=db.as_retriever().get_relevant_documents(query)[0].page_content, question=query)
    )
    return response #Corrected to return the response

# 6. Example usage
query = "What are the skills of candidates in the Finance category?"
result = run_query(query)
print(result)

In [ ]:
###Prompt Engineering 3:  Llama 2 ###

from langchain.chains import RetrievalQA
from langchain.llms import LlamaCpp

# 1. Load the FAISS vectorstore
db = FAISS.load_local("more_resumes_faiss", embeddings)

# 2. Initialize Llama 2
# Replace with the path to your Llama 2 model file
llm = LlamaCpp(model_path="/path/to/your/llama-2-7b-chat.ggmlv3.q4_0.bin")


# 3. Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

# 4. Define your prompt template
prompt_template = """
Use the following pieces of context to answer the question at the end.
If you don't have enough information to answer the question, just say "I don't know".

Context:
{context}

Question:
{question}
"""

# 5. Function to run a query
def run_query(query):
    """Runs a query against the resume dataset using prompt engineering."""
    response = qa_chain.run(
        prompt_template.format(context=db.as_retriever().get_relevant_documents(query)[0].page_content, question=query)
    )
    return response #Corrected to return the response

# 6. Example usage
query = "What are the skills of candidates in the Finance category?"
result = run_query(query)
print(result)

# Testing Code with Redacted PII data

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
#from docx import Document
import os
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pdfplumber
import nltk
nltk.download('punkt')
from textblob import TextBlob
import plotly.graph_objects as go
import plotly.express as px
import os
import json
#from docx import Document
import pdfplumber
from bs4 import BeautifulSoup
import spacy
import matplotlib.pyplot as plt
from wordcloud import WordCloud
#spacy.cli.download("en_core_web_sm")
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from collections import Counter
from plotly.offline import plot
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dezri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Redacted with spacy

# adding Genesis Code
- dataset csv name = spacy_redacted_documents_with_id_and_category.csv
https://python.langchain.com/docs/introduction/

In [ ]:
#pip install langchain

In [ ]:
#pip install langchain-core

In [ ]:
#pip install langchain-community

In [ ]:
#pip install langchain-openai

In [ ]:
#pip install langchain-huggingface

In [ ]:
#pip install --upgrade pydantic

In [ ]:
#pip install tf-keras

In [ ]:
#pip install langchain langchain-huggingface tf-keras

In [3]:
pip install keras==2.11.0

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --upgrade tensorflow keras

  Obtaining dependency information for keras from https://files.pythonhosted.org/packages/c2/88/eef50051a772dcb4433d1f3e4c1d6576ba450fe83e89d028d7e8b85a2122/keras-3.6.0-py3-none-any.whl.metadata
  Using cached keras-3.6.0-py3-none-any.whl.metadata (5.8 kB)
Using cached keras-3.6.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Uninstalling keras-2.11.0:
      Successfully uninstalled keras-2.11.0
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install faiss-cpu

  Obtaining dependency information for faiss-cpu from https://files.pythonhosted.org/packages/87/2b/850200d901fd0409232d9bcae26228ab9aadf4803bbcc38f93d6f81cab37/faiss_cpu-1.9.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.9 MB 640.0 kB/s eta 0:00:24
   ---------------------------------------- 0.1/14.9 MB 787.7 kB/s eta 0:00:19
    --------------------------------------- 0.2/14.9 MB 1.8 MB/s eta 0:00:09
   - -------------------------------------- 0.5/14.9 MB 3.5 MB/s eta 0:00:05
   -- ------------------------------------- 0.7/14.9 MB 3.9 MB/s eta 0:00:04
   --- ------------------------------------ 1.2/14.9 MB 4.9 MB/s eta 0:00:03
   ---- ----------------------------------- 1.6/14.9 MB 5.9 MB/s eta 0:00:03
   ---- ----------------------------------- 1.9/14.9 MB 6.2 MB/s eta 0:00:03
   ------ ------------------

In [2]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from datasets import load_dataset
import keras

In [4]:
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

# SENTENCE TRANSFORMER MODELS
For a recommender system that **matches job descriptions to resumes**, you’ll want an LLM with strong natural language understanding and embeddings capabilities

cost-effective, open-source solution - 
SBERT would allow you to keep everything in-house and potentially run it more efficiently.

- https://www.sbert.net/index.html

The all-* models were trained on all available training data (more than 1 billion training pairs) and are designed as general purpose models. 

Pretrained models
 - https://www.sbert.net/docs/sentence_transformer/pretrained_models.html
 
 all-mpnet-base-v2 - https://huggingface.co/sentence-transformers/all-mpnet-base-v2
   - Model insight: "all-mpnet-base-v2" is used to convert text (in this case, resume content) into numerical vectors called embeddings.
   - Functionality: Embeddings capture the semantic meaning of the text, allowing you to compare the similarity between different pieces of text based on their vector representations.
 
all-distilroberta-v1 - https://huggingface.co/sentence-transformers/all-distilroberta-v1

all-MiniLM-L12-v2 - https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2

In [11]:

# 1. Load the Hugging Face Embeddings model

embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")

# 2. Load the more_resumes dataset

redacted_resumes = pd.read_csv('spacy_redacted_documents_with_id_and_category.csv')
redacted_resumes = redacted_resumes.head(5) #use to limit dataset for testing

# 3. Split resumes into smaller chunks for embedding

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = []
for index, resume in more_resumes.iterrows(): 
    texts = text_splitter.split_text(resume['Redacted Text'])  
    docs.extend([Document(page_content=t, metadata={'source': resume['Category']}) for t in texts]) 
# 4. Embed the resume chunks and store them in a FAISS vectorstore

db = FAISS.from_documents(docs, embeddings)
db.save_local("redacted_resumes_faiss")


# Requires Login and API Keys - Free Trails May be Available

Groq API Instructions - https://docs.aicontentlabs.com/articles/groq-api-key/
Cohere Instructions - https://docs.aicontentlabs.com/articles/cohere-api-key/
Open AI Instructions - https://docs.aicontentlabs.com/articles/openai-api-key/

In [18]:
###Prompt Engineering 1: Open AI's GPT 4 ###

from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import os        #for API Key
import openai


# 1. Load the FAISS vectorstore
db = FAISS.load_local("redacted_resumes_faiss", embeddings, allow_dangerous_deserialization=True)

# 2. Initialize OpenAI's GPT-4
# Make sure you set your OPENAI_API_KEY environment variable

llm = OpenAI(model_name="gpt-4", temperature=0)

# Set's GR's API key as an environment variable (recommended)
os.environ["genesis_gpt4_API_key"] = "genesis_gpt4_API_key" #Replace YOUR_API_KEY with your actual OpenAI key

# 3. Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

# 4. Define your prompt template
prompt_template = """
Use the following pieces of context to answer the question at the end.
If you don't have enough information to answer the question, just say "I don't know".

Context:
{context}

Question:
{question}
"""

# 5. Function to run a query
def run_query(query):
    """Runs a query against the resume dataset using prompt engineering."""
    response = qa_chain.run(
        prompt_template.format(context=db.as_retriever().get_relevant_documents(query)[0].page_content, question=query)
    )
    return response #Corrected to return the response

# 6a. Example usage 1
query = "Based on this resume, what are the top jobs I can qualify for?"
result = run_query(query)
print(result)

# 6b. Example usage 2
query1 = "I live in a high cost city and need to make at least $80k salary per year, based on my resume, what jobs should I apply for to get me this salary?"
result1 = run_query(query1)
print(result1)

C:\Users\dezri\anaconda3\Lib\site-packages\langchain_community\llms\openai.py:255: UserWarning:

You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`



ValidationError: 1 validation error for OpenAIChat
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_kwargs': {'model_...allowed_special': 'all'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

In [ ]:
pip install cohere

In [17]:
###Prompt Engineering 2:  Cohere ###

from langchain.chains import RetrievalQA
from langchain.llms import Cohere

# 1. Load the FAISS vectorstore
db = FAISS.load_local("redacted_resumes_faiss", embeddings, allow_dangerous_deserialization=True)

# 2. Initialize Cohere
# Make sure you set your COHERE_API_KEY environment variable
llm = Cohere(model="command-xlarge-nightly", temperature=0)

# 3. Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

# 4. Define your prompt template
prompt_template = """
Use the following pieces of context to answer the question at the end.
If you don't have enough information to answer the question, just say "I don't know".

Context:
{context}

Question:
{question}
"""

# 5. Function to run a query
def run_query(query):
    """Runs a query against the resume dataset using prompt engineering."""
    response = qa_chain.run(
        prompt_template.format(context=db.as_retriever().get_relevant_documents(query)[0].page_content, question=query)
    )
    return response #Corrected to return the response

# 6. Example usage
query = "What are the skills of candidates in the HR category?"
result = run_query(query)
print(result)

ValidationError: 1 validation error for Cohere
  Value error, Did not find cohere_api_key, please add an environment variable `COHERE_API_KEY` which contains it, or pass `cohere_api_key` as a named parameter. [type=value_error, input_value={'model': 'command-xlarge...None, 'max_retries': 10}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

In [21]:
pip install llama-cpp-python

     ---------------------------------------- 0.0/63.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/63.9 MB ? eta -:--:--
     --------------------------------------- 0.0/63.9 MB 435.7 kB/s eta 0:02:27
     --------------------------------------- 0.1/63.9 MB 525.1 kB/s eta 0:02:02
     ---------------------------------------- 0.2/63.9 MB 1.3 MB/s eta 0:00:49
     ---------------------------------------- 0.3/63.9 MB 2.0 MB/s eta 0:00:32
     ---------------------------------------- 0.7/63.9 MB 3.7 MB/s eta 0:00:18
      --------------------------------------- 1.0/63.9 MB 4.4 MB/s eta 0:00:15
      --------------------------------------- 1.4/63.9 MB 5.1 MB/s eta 0:00:13
     - -------------------------------------- 1.7/63.9 MB 5.6 MB/s eta 0:00:12
     - -------------------------------------- 2.2/63.9 MB 6.3 MB/s eta 0:00:10
     - -------------------------------------- 2.6/63.9 MB 6.7 MB/s eta 0:00:10
     - -------------------------------------- 3.0/63.9 MB 7.1 MB

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [20 lines of output]
  *** scikit-build-core 0.10.7 using CMake 3.30.5 (wheel)
  *** Configuring CMake...
  2024-11-01 07:09:55,603 - scikit_build_core - WARNING - Can't find a Python library, got libdir=None, ldlibrary=None, multiarch=None, masd=None
  loading initial cache file C:\Users\dezri\AppData\Local\Temp\tmp5e7c_rdr\build\CMakeInit.txt
  -- Building for: NMake Makefiles
  CMake Error at CMakeLists.txt:3 (project):
    Running
  
     'nmake' '-?'
  
    failed with:
  
     no such file or directory
  
  
  CMake Error: CMAKE_C_COMPILER not set, after EnableLanguage
  CMake Error: CMAKE_CXX_COMPILER not set, after EnableLanguage
  -- Configuring incomplete, errors occurred!
  
  *** CMake configuration failed
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for llama-cpp-python
ERROR: Could not build wheels for llama-cpp-python,

In [20]:
###Prompt Engineering 3:  Llama 2 ###

from langchain.chains import RetrievalQA
from langchain.llms import LlamaCpp

# 1. Load the FAISS vectorstore
db = FAISS.load_local("redacted_resumes_faiss", embeddings, allow_dangerous_deserialization=True)

# 2. Initialize Llama 2
# Replace with the path to your Llama 2 model file
llm = LlamaCpp(model_path="/path/to/your/llama-2-7b-chat.ggmlv3.q4_0.bin")


# 3. Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

# 4. Define your prompt template
prompt_template = """
Use the following pieces of context to answer the question at the end.
If you don't have enough information to answer the question, just say "I don't know".

Context:
{context}

Question:
{question}
"""

# 5. Function to run a query
def run_query(query):
    """Runs a query against the resume dataset using prompt engineering."""
    response = qa_chain.run(
        prompt_template.format(context=db.as_retriever().get_relevant_documents(query)[0].page_content, question=query)
    )
    return response #Corrected to return the response

# 6. Example usage
query = "What are the skills of candidates in the Finance category?"
result = run_query(query)
print(result)

ImportError: Could not import llama-cpp-python library. Please install the llama-cpp-python library to use this embedding model: pip install llama-cpp-python